In [1]:
import os
import glob
import geemap
import ee, datetime
from IPython.display import Image
import pandas as pd
import geopandas as gpd
from pylab import *
import seaborn as sns
import folium
import matplotlib.pyplot as plt
%matplotlib inline
import altair as alt

In [2]:
# Setting up Google Earth Engine (GEE)
# Getting tokens and activate EE
ee.Authenticate()

# Start up this session
ee.Initialize()

Enter verification code: 4/1AWtgzh5cNbVdO35eVvD5xltiDFG6TM3NzNYKZMUGTDFwpH-VzsXqZqFtF7U

Successfully saved authorization token.


## Masking clouds

### Sentinel 2

In [3]:
# Using QA band to filter clouds
def maskS2clouds(image):
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0)
  mask2 = qa.bitwiseAnd(cirrusBitMask).eq(0)
  mask3 = qa.bitwiseAnd(cloudBitMask).eq(0).bitwiseAnd(cirrusBitMask).eq(0)

  return image.updateMask(mask).updateMask(mask2)

### Landsat 8

In [4]:
def maskL8clouds(image):
  qa = image.select('pixel_qa')

  # Bits 3 and 5 are clouds and cloud shadows.
  cloudBitMask = 1 << 3
  cloudShadowBitMask = 1 << 5

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0)
  mask2 = qa.bitwiseAnd(cloudShadowBitMask).eq(0)
  mask3 = qa.bitwiseAnd(cloudBitMask).eq(0).bitwiseAnd(cloudShadowBitMask).eq(0)

  return image.updateMask(mask).updateMask(mask2)

### Landsat 7

In [5]:
def maskL7clouds(image):
  qa = image.select('pixel_qa')

  # Bits 4 and 6 are clouds and cloud shadows.
  cloudBitMask = 1 << 4
  cloudShadowBitMask = 1 << 6

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0)
  mask2 = qa.bitwiseAnd(cloudShadowBitMask).eq(0)
  mask3 = qa.bitwiseAnd(cloudBitMask).eq(0).bitwiseAnd(cloudShadowBitMask).eq(0)

  return image.updateMask(mask).updateMask(mask2)

# NDVI Bands Setup

### Sentinel 2

In [6]:
# NDVI for Sentine 2 bands
def senNDVI(image):
    return image.normalizedDifference(['B8', 'B4'])

### Landsat 8

In [7]:
# NDVI for Landsat 8 bands
def l8NDVI(image):
    return image.normalizedDifference(['B5', 'B4'])

### Landsat 7

In [8]:
# NDVI for Landsat 7 bands
def l7NDVI(image):
    return image.normalizedDifference(['B4', 'B3'])

# Set study area

In [9]:
# Rim Fire, CA
coords = [-119.634, 37.824]

# Create earth engine point
point = {'type':'Point', 'coordinates':coords}

# Time Period: Before Fire and its NDVI Analysis

In [10]:
# Set start and end date
startTime = datetime.datetime(2012, 1, 1)
endTime = datetime.datetime(2012, 9, 30)

In [11]:
# Getting L7 image collection
collection = ee.ImageCollection("LANDSAT/LE07/C01/T2").filterDate(startTime, endTime).filterBounds(point)

In [12]:
# Limit the collection to the 1 most recent images.
recent = collection.sort('system:time_start', False).limit(1)
# Pull the image from the collection
image = recent.first()
print('Recent images: ' + str(recent.getInfo()) + '\n')

Recent images: {'type': 'ImageCollection', 'bands': [], 'id': 'LANDSAT/LE07/C01/T2', 'version': 1648901714075430.0, 'properties': {'type_name': 'ImageCollection', 'keywords': ['global', 'landsat', 'usgs'], 'visualization_1_bands': 'B4,B3,B2', 'thumb': 'https://mw1.google.com/ges/dd/images/LANDSAT_RAW_thumb.png', 'description': '<p>empty</p><p><b>Provider: <a href="https://landsat.usgs.gov/">USGS</a></b><br><p><b>Revisit Interval</b><br>\n  16 days\n</p><p><b>Bands</b><table class="eecat"><tr><th scope="col">Name</th><th scope="col">Pixel Size</th><th scope="col">Wavelength</th><th scope="col">Description</th></tr><tr><td>B1</td><td>\n      30 meters\n</td><td>0.45 - 0.52 µm</td><td><p>Blue</p></td></tr><tr><td>B2</td><td>\n      30 meters\n</td><td>0.52 - 0.60 µm</td><td><p>Green</p></td></tr><tr><td>B3</td><td>\n      30 meters\n</td><td>0.63 - 0.69 µm</td><td><p>Red</p></td></tr><tr><td>B4</td><td>\n      30 meters\n</td><td>0.77 - 0.90 µm</td><td><p>Near infrared</p></td></tr><tr><t

In [13]:
# Limit the collection to the 1 most recent images.
recent = collection.sort('system:time_start', False).limit(1)
# Pull the image from the collection
image = recent.first()

In [14]:
# Set the visualization parameters for the output
visParams = {'bands': ['B3', 'B2', 'B1'], 'max': 255, 'dimensions': 1000}

# Create image thumbnail in earth engine and view
thumbnail = image.getThumbUrl(visParams)
Image(url = thumbnail)

In [15]:
# Get the timestamp and convert it to a date.
date = ee.Date(image.get('system:time_start')).getInfo()
x = date['value']
print(datetime.datetime.fromtimestamp(x/1000))

2012-09-21 11:35:23.870000


In [16]:
# Get a list of all metadata properties
properties = collection.propertyNames()
print('Metadata properties: '+str(properties.getInfo())) 

Metadata properties: ['type_name', 'keywords', 'visualization_1_bands', 'thumb', 'description', 'source_tags', 'visualization_1_name', 'system:id', 'title', 'visualization_0_gain', 'system:visualization_0_gain', 'product_tags', 'system:visualization_2_gain', 'visualization_1_gain', 'provider', 'system:visualization_2_name', 'system:version', 'system:visualization_1_bands', 'visualization_0_name', 'date_range', 'visualization_2_bands', 'visualization_2_name', 'period', 'system:visualization_0_bands', 'visualization_2_gain', 'provider_url', 'sample', 'system:visualization_1_name', 'tags', 'system:visualization_2_bands', 'system:visualization_0_name', 'system:visualization_1_gain', 'visualization_0_bands']


In [17]:
# Get the 'WRS_ROW' location identifier
target_row = 'WRS_ROW'
meta_row = image.get(target_row).getInfo()
print(f'{target_row} = {meta_row}')

# Get the 'WRS_PATH' location identifier
target_path = 'WRS_PATH'
meta_path = image.get(target_path).getInfo()
print(f'{target_path} = {meta_path}')

# List of data fields in the image
properties = image.propertyNames()
print('Metadata properties: '+str(properties.getInfo())) 

# Get the scene id
target = 'LANDSAT_SCENE_ID'
meta = image.get(target).getInfo()
print(f'{target} = {meta}')

WRS_ROW = 34
WRS_PATH = 43
Metadata properties: ['RADIANCE_MULT_BAND_5', 'IMAGE_QUALITY', 'RADIANCE_MULT_BAND_3', 'RADIANCE_MULT_BAND_4', 'RADIANCE_MULT_BAND_1', 'RADIANCE_MULT_BAND_2', 'system:id', 'CORRECTION_GAIN_BAND_6_VCID_2', 'CORRECTION_GAIN_BAND_6_VCID_1', 'CORRECTION_GAIN_BAND_2', 'CORRECTION_GAIN_BAND_1', 'system:footprint', 'REFLECTIVE_SAMPLES', 'CORRECTION_GAIN_BAND_8', 'CORRECTION_GAIN_BAND_7', 'CORRECTION_GAIN_BAND_5', 'SUN_AZIMUTH', 'CORRECTION_GAIN_BAND_4', 'CORRECTION_GAIN_BAND_3', 'CPF_NAME', 'GAIN_CHANGE_BAND_7', 'GAIN_CHANGE_BAND_8', 'GAIN_CHANGE_BAND_5', 'DATE_ACQUIRED', 'GAIN_CHANGE_BAND_3', 'ELLIPSOID', 'GAIN_CHANGE_BAND_4', 'GAIN_CHANGE_BAND_1', 'GAIN_CHANGE_BAND_2', 'RADIANCE_ADD_BAND_6_VCID_1', 'RADIANCE_ADD_BAND_6_VCID_2', 'SENSOR_MODE', 'STATION_ID', 'RESAMPLING_OPTION', 'ORIENTATION', 'WRS_ROW', 'GAIN_BAND_6_VCID_2', 'RADIANCE_MULT_BAND_7', 'GAIN_BAND_6_VCID_1', 'RADIANCE_MULT_BAND_8', 'CLOUD_COVER', 'COLLECTION_CATEGORY', 'GRID_CELL_SIZE_REFLECTIVE', 'DATA

In [18]:
# Estimating the cloud cover
target = 'CLOUD_COVER'
meta = image.get(target).getInfo()
print(f'{target} = {meta}')

CLOUD_COVER = 8


#### Calculating NDVI

In [19]:
# Run the Landsat 7 NDVI function
l7ndvi = l7NDVI(image)

# Change visualization setting for the NDVI data
visParams = {'min':0, 'max': 1, 'dimensions':1000}
thumbnail = l7ndvi.getThumbUrl(visParams)
Image(url=thumbnail)

#### Making an interactive NDVI graph

In [20]:
# Visualization settings
rgbParams = {'bands': ['B4', 'B3', 'B2'], 'max': 3000, 'gamma': [1.4, 1.2, 1.4]}
ndviParams = {'min':0, 'max': 1}

# Create the map
Map = geemap.Map(location=coords[::-1], zoom = 10)

# Add a marker to show the location
Map.add_marker(coords[::-1])

# Add the Landsat 7 image to the map
Map.addLayer(image, rgbParams, 'Natural Color Landsat 7');

# Add the NDVI image to the map
Map.addLayer(l7ndvi, ndviParams, 'NDVI')

# Add layer control
Map.addLayerControl()

# Display the map
Map

Map(center=[37.824, -119.634], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chi…

# Time Period: During Fire and its NDVI Analysis¶

In [23]:
# Set start and end date
#startTime = datetime.datetime(2013, 8, 1)
#endTime = datetime.datetime(2012, 10, 31)

In [24]:
# Create Sentinel-2 image collection
collection = ee.ImageCollection("COPERNICUS/S2").filterDate(startTime, endTime).filterBounds(point)

# Time Period: After Fire and its NDVI Analysis¶

In [22]:
# Set start and end date
#startTime = datetime.datetime(2020, 1, 1)
#endTime = datetime.datetime(2020, 12, 31)